In [2]:
import shapefile
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

# 读取矢量文件
sf = shapefile.Reader("D:\Lenovo\Desktop\云南大学\大创\复杂地形条件对空间关系构建的影响研究项目\data\构建权重矩阵示例数据\CTCI_墨卡托投影.shp")

# 获取字段X、Y、Z和ID的索引
fields = [field[0] for field in sf.fields[1:]]
x_index = fields.index('X')
y_index = fields.index('Y')
z_index = fields.index('Z')
id_index = fields.index('ID')

# 获取点坐标和ID
points = []
ids = []
for sr in sf.shapeRecords():
    points.append((sr.record[x_index], sr.record[y_index], sr.record[z_index]))
    ids.append(sr.record[id_index])

# 转换为numpy数组
data = np.array(points)

# 设置距离阈值
threshold = 300000  # 单位：米

# 计算空间权重矩阵
num_points = len(ids)
weight_matrix = np.zeros((num_points, num_points))

for i in range(num_points):
    center_point = data[i]
    distances = cdist([center_point], data)[0]
    weights = np.exp(-distances**2 / (2 * threshold**2))
    weight_matrix[i] = weights

# 创建DataFrame并设置索引和列名
df_weights = pd.DataFrame(weight_matrix, index=ids, columns=ids)

# 保存空间权重矩阵为csv文件
df_weights.to_csv('D:\Lenovo\Desktop\云南大学\大创\程序代码\空间权重矩阵测试\高斯权函数矩阵.csv')